<center> <h1> COVID-19 Analysis Dashboard</h1> </center>
![Coronavirus particle Image](https://www.apta.com/wp-content/uploads/home-banner-1.jpg)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
from IPython.core.display import display,HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import folium

In [2]:
death_df=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
#confirmed_df.head()

In [4]:
#recovered_df.head()

In [5]:
#confirmed_df.head()

In [6]:
#country_df.head()

In [7]:
country_df.columns=map(str.lower, country_df.columns)

In [8]:
recovered_df.columns=map(str.lower,recovered_df.columns)
death_df.columns=map(str.lower,death_df.columns)
confirmed_df.columns=map(str.lower,confirmed_df.columns)

In [9]:
confirmed_df=confirmed_df.rename(columns={'province/state':'state','country/region':'country'})
country_df=country_df.rename(columns={'country_region':'country'})
recovered_df=recovered_df.rename(columns={'province/state':'state','country/region':'country'})
death_df=death_df.rename(columns={'province/state':'state','country/region':'country'})

In [10]:
confirmed_total=int(country_df['confirmed'].sum())
deaths_total=int(country_df['deaths'].sum())
recovered=int(country_df['recovered'].sum())
recovered

646012

In [11]:
display(HTML("<div style = 'background-color:#504e4e; padding:30px '>"+
             "<span style = 'color:#fff; font-size: 30px;'> Confirmed: " + str(confirmed_total)+"</span>" + 
             "<span style = 'color: red; font-size: 30px;'> Deaths: "+ str(deaths_total)+"</span>"+
             "<span style= 'color:lightgreen; font-size:30px;'> Recovered: "+str(recovered)+"</span>"+
             "</div>"
            )
       )

## COVID-19 Confirmed/Death/Recovered cases by countries

#### Enter number of countries you want the data for

In [12]:
fig=go.FigureWidget(layout=go.Layout())
def highlight_col(x):
    r = 'background-color: red'
    g =  'background-color: gray'
    l = 'background-color: lightgreen'
    df1=pd.DataFrame('',index=x.index, columns=x.columns)
    df1.iloc[:,4]=g
    df1.iloc[:,5]=r
    df1.iloc[:,6]=l
    
    return df1

def show_latest_cases(n):
    n=int(n)
    return country_df.sort_values('confirmed',ascending=False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout=widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig],layout=ipywLayout)
    
    

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [13]:
sorted_country_df=country_df.sort_values('confirmed', ascending=False)

## Worst Hit Countries
#### Slide to check for the worst hit countries

In [14]:
def bubble_chart(n):
    fig =  px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
                     hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n)+" Worst Hit Countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width=700
    )
    fig.show();

interact(bubble_chart,n=10)

ipywLayout=widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

## 10 Worst Hit Countries - Confirmed Cases

In [15]:
px.bar(
    sorted_country_df.head(10),
    x="country",
    y="confirmed",
    title="Top 10 worst Hit Countries",
    color_discrete_sequence=["red"],
    height=500,
    width=800
    
)

## 10 Worst Hit Countries - Deaths

In [16]:
px.bar(
    sorted_country_df.head(10),
    x="country",
    y="deaths",
    title="Top 10 countries with deaths",
    color_discrete_sequence=["red"],
    height=500,
    width=800
)

## 10 Worst Hit Countries - Recovered Cases

In [17]:
px.bar(
    sorted_country_df.head(10),
    x="country",
    y="recovered",
    title="Top 10 countries with recovery",
    color_discrete_sequence=["red"],
    height=500,
    width=800
)

## Global spread of COVID-19

In [18]:
world_map = folium.Map(location=[1,0], tiles="openstreetmap", zoom_start=2, max_zoom=6, min_zoom=2)
for i in range(0, len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip="<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                    "</ul></div>",
         ).add_to(world_map)
world_map